In [146]:
from typing import Dict, Optional, Tuple, Union, Callable

import clingo
import clingo.ast
from clingo.ast import ProgramBuilder

In [147]:
program = """

head1(X,Y,Z) :- body1(X), body2(Y), body3(Z).

head2(X) :- body1(X), body2(X-1).

head3(X) :- body4(X).

"""

In [148]:
def get_parsed_program(program: str):
    nodes = []
    clingo.ast.parse_string(program, lambda stm: nodes.append(stm))
    return nodes

In [149]:
nodes = get_parsed_program(program)
nodes

[ast.Program(Location(begin=Position(filename='<string>', line=1, column=1), end=Position(filename='<string>', line=1, column=1)), 'base', []),
 ast.Rule(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=46)), ast.Literal(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 'head1', [ast.Variable(Location(begin=Position(filename='<string>', line=3, column=7), end=Position(filename='<string>', line=3, column=8)), 'X'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=9), end=Position(filename='<string>', line=3, column=10)), 'Y'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=11), end=Position(filename='<string>', line=3, column=12)), 'Z')], 0))), [ast.Literal(

In [150]:
pos = clingo.ast.Position('<string>', 0, 0)
loc = clingo.ast.Location(pos, pos)

In [151]:
Substitute = Union[clingo.ast.AST, Callable[[clingo.ast.AST], clingo.ast.AST], str, clingo.Symbol]


def substitute(node: clingo.ast.AST, sub: Substitute):
    if isinstance(sub, clingo.ast.AST):
        print('Substituting with AST')
        return sub
    elif callable(sub):
        print('Substituting with call')
        return sub(node)
    elif isinstance(sub, str):
        if node.ast_type is clingo.ast.ASTType.Variable:
            print('Substituting with new variable')
            return clingo.ast.Variable(loc, sub)
        elif node.ast_type is clingo.ast.ASTType.Function:
            print('Substituting with new function')
            return clingo.ast.Function(loc, sub, node.arguments, node.external)
    else:
        assert isinstance(sub, clingo.Symbol)
        print('Substituting with new term')
        return clingo.ast.SymbolicTerm(loc, sub)


class SubstitutionTransformer(clingo.ast.Transformer):

    def __init__(self,
                 var_env: Optional[Dict[str, Substitute]] = None,
                 func_env: Optional[Dict[Tuple[str, int], Substitute]] = None,
                 symbol_env: Optional[Dict[clingo.Symbol, Substitute]] = None):
        self.__var_env = var_env or {}
        self.__func_env = func_env or {}
        self.__symbol_env = symbol_env or {}

    def visit_Variable(self, variable: clingo.ast.AST,
                       var_env: Optional[Dict[str, Substitute]] = None,
                       func_env: Optional[Dict[Tuple[str, int], Substitute]] = None,
                       symbol_env: Optional[Dict[clingo.Symbol, Substitute]] = None):
        _var_env = self.__var_env | (var_env or {})
        print('Visiting', variable, variable.ast_type.name)
        if variable.name in _var_env:
            print('Substituting', variable.name, 'with', _var_env[variable.name])
            return substitute(variable, _var_env[variable.name])
        return variable

    def visit_SymbolicTerm(self, term: clingo.ast.AST,
                           var_env: Optional[Dict[str, Substitute]] = None,
                           func_env: Optional[Dict[Tuple[str, int], Substitute]] = None,
                           symbol_env: Optional[Dict[clingo.Symbol, Substitute]] = None):
        _symbol_env = self.__symbol_env | (symbol_env or {})
        if term in _symbol_env:
            return substitute(term, _symbol_env[term.symbol])
        return term

    def visit_Function(self, function: clingo.ast.AST,
                       var_env: Optional[Dict[str, Substitute]] = None,
                       func_env: Optional[Dict[Tuple[str, int], Substitute]] = None,
                       symbol_env: Optional[Dict[clingo.Symbol, Substitute]] = None):
        print(function)
        function = function.update(**self.visit_children(function, var_env=var_env, func_env=func_env, symbol_env=symbol_env))
        print(function)
        _func_env = self.__func_env | (func_env or {})
        for func in _func_env:
            name, arity = func
            if function.name == name and len(function.arguments) == arity:
                return substitute(function, _func_env[func])
        return function



In [152]:
st = SubstitutionTransformer()

In [153]:
func_env = {
    ('head1', 3): 'headA'
}
var_env = {
    'X': 'A'
}

In [154]:
nodes = []

clingo.ast.parse_string(program, lambda stm: nodes.append(st.visit(stm, var_env=var_env, func_env=func_env)))
nodes

head1(X,Y,Z)
Visiting X Variable
Substituting X with A
Substituting with new variable
Visiting Y Variable
Visiting Z Variable
head1(A,Y,Z)
Substituting with new function
body1(X)
Visiting X Variable
Substituting X with A
Substituting with new variable
body1(A)
body2(Y)
Visiting Y Variable
body2(Y)
body3(Z)
Visiting Z Variable
body3(Z)
head2(X)
Visiting X Variable
Substituting X with A
Substituting with new variable
head2(A)
body1(X)
Visiting X Variable
Substituting X with A
Substituting with new variable
body1(A)
body2((X-1))
Visiting X Variable
Substituting X with A
Substituting with new variable
body2((A-1))
head3(X)
Visiting X Variable
Substituting X with A
Substituting with new variable
head3(A)
body4(X)
Visiting X Variable
Substituting X with A
Substituting with new variable
body4(A)


[ast.Program(Location(begin=Position(filename='<string>', line=1, column=1), end=Position(filename='<string>', line=1, column=1)), 'base', []),
 ast.Rule(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=46)), ast.Literal(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=0, column=0), end=Position(filename='<string>', line=0, column=0)), 'headA', [ast.Variable(Location(begin=Position(filename='<string>', line=0, column=0), end=Position(filename='<string>', line=0, column=0)), 'A'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=9), end=Position(filename='<string>', line=3, column=10)), 'Y'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=11), end=Position(filename='<string>', line=3, column=12)), 'Z')], 0))), [ast.Literal(L

In [155]:
print('\n'.join(map(str, nodes)))

#program base.
headA(A,Y,Z) :- body1(A); body2(Y); body3(Z).
head2(A) :- body1(A); body2((A-1)).
head3(A) :- body4(A).


In [156]:
new_ctl = clingo.Control()
new_ctl.configuration.solve.models = 0

In [157]:
with ProgramBuilder(new_ctl) as pb:
    for node in nodes:
        pb.add(node)

In [158]:
new_ctl.ground([('base', [])])

<string>:3:17-25: info: atom does not occur in any rule head:
  body1(A)

<string>:3:27-35: info: atom does not occur in any rule head:
  body2(Y)

<string>:3:37-45: info: atom does not occur in any rule head:
  body3(Z)

<string>:5:13-21: info: atom does not occur in any rule head:
  body1(A)

<string>:5:23-33: info: atom does not occur in any rule head:
  body2((A+-1))

<string>:7:13-21: info: atom does not occur in any rule head:
  body4(A)

